## Imports & display options

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from IPython.display import display

# Optional: nicer DataFrame display
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Config & data loading

In [51]:
file_path = "data.csv"
target_column = 'DDG'
indexes_to_remove = [] 

# Read CSV
df = pd.read_csv(file_path)

# Drop specified rows
df_cleaned = df.drop(index=indexes_to_remove)


## Target/feature split & basic cleaning

In [52]:
# Target
Y = df_cleaned[target_column]

# Features: drop target and identifier (if present)
X = df_cleaned.drop(columns=[target_column, 'Folder_Name'], errors='ignore')

# Fill NaNs with column means, then drop any columns still containing NaNs
X_filled = X.fillna(X.mean(numeric_only=True))
X_filled = X_filled.dropna(axis=1)

# Remove near-constant features
X_cleaned = X_filled.loc[:, X_filled.var() > 0.01]

X_cleaned.shape
X_cleaned.to_csv('X_cleaned.csv', index=False)

## Correlation pruning

In [53]:
corr = X_cleaned.corr(method='pearson').abs()
upper = np.triu(np.ones(corr.shape), k=1)

high_i, high_j = np.where((corr.values > 0.85) & (upper == 1))
drop_features = []
cols = corr.columns

for i, j in zip(high_i, high_j):
    fi, fj = cols[i], cols[j]
    if 'buried_volume' in fj:
        drop_features.append(fi)
    elif 'buried_volume' in fi:
        drop_features.append(fj)
    elif '_bol' in fj:
        drop_features.append(fi)
    elif '_bol' in fi:
        drop_features.append(fj)
    else:
        drop_features.append(fj)

drop_features = list(set(drop_features))
X_reduced = X_cleaned.drop(columns=drop_features, errors='ignore')

X_reduced.shape

(252, 130)

## Scaling & PCA-derived top-20 features

In [68]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Assumes X_reduced is a pandas DataFrame of features (defined earlier)

# Standardize features before PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

# Fit PCA and retain components that explain 95% of variance
pca = PCA(n_components=0.95)
pca.fit(X_scaled)

# PCA loadings (components) and explained variance ratio per component
loadings = pca.components_                    # shape: (n_components, n_features)
explained = pca.explained_variance_ratio_     # shape: (n_components,)

# Compute weighted squared loadings as feature importance
feature_importance = np.sum(explained.reshape(-1, 1) * (loadings ** 2), axis=0)

# Sort features by importance (descending) and build importance DataFrame with ranks
sorted_idx = np.argsort(feature_importance)[::-1]
important_features = X_reduced.columns[sorted_idx]
feature_importance_sorted = feature_importance[sorted_idx]

pca_importance_df = pd.DataFrame({
    'Descriptor': important_features,
    'PCA_Importance_Score': feature_importance_sorted
})
pca_importance_df['Rank'] = range(1, len(pca_importance_df) + 1)

# Select top-20 features and create dataset containing only those features
top_20_importance_df = pca_importance_df.head(20)
top_20_features = top_20_importance_df['Descriptor'].tolist()
X_PCA_top_20 = X_reduced[top_20_features]


## Lasso- and RF-based feature selection

In [69]:
# LassoCV selection
lasso = LassoCV(cv=5, max_iter=10000, random_state=42)
lasso.fit(X_scaled, Y)
lasso_abs = np.abs(lasso.coef_)
top_20_features_lasso = X_reduced.columns[np.argsort(lasso_abs)[::-1][:20]]
X_LASSO_top_20 = X_reduced[top_20_features_lasso]

# Random Forest importance
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_reduced, Y)
rf_imp = rf.feature_importances_
top_20_features_rf = X_reduced.columns[np.argsort(rf_imp)[::-1][:20]]
X_RF_top_20 = X_reduced[top_20_features_rf]

# Combined (Lasso ∪ RF), deduplicated
X_combined = pd.concat([X_RF_top_20, X_LASSO_top_20], axis=1)
X_combined = X_combined.loc[:, ~X_combined.columns.duplicated()]

X_LASSO_top_20.shape, X_RF_top_20.shape, X_combined.shape

((252, 20), (252, 20), (252, 37))

## Train/test split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)

# Refit scaler on the training slice for fair evaluation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape

((201, 37), (51, 37))

## Define regressors

In [71]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

models = {
    'Linear Regression': LinearRegression(),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42),
    'Random Forest': RandomForestRegressor(
        n_estimators=50, max_depth=4, min_samples_split=2, min_samples_leaf=1,
        bootstrap=True, random_state=42
    ),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(
        n_estimators=50, max_depth=4, learning_rate=0.2,
        subsample=0.8, colsample_bytree=0.4, random_state=42
    ),
    'Lasso Regression': Lasso(alpha=0.1)
}

list(models.keys())

['Linear Regression',
 'ElasticNet',
 'Random Forest',
 'AdaBoost',
 'XGBoost',
 'Lasso Regression']

## Train, cross-validate, and evaluate

In [72]:
results = []

for model_name, model in models.items():
    # 5-fold CV on training set (neg MSE -> MSE)
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train, cv=5,
        scoring='neg_mean_squared_error', n_jobs=-1
    )
    mean_cv_mse = -cv_scores.mean()

    # Fit
    model.fit(X_train_scaled, y_train)

    # Predict
    y_tr_pred = model.predict(X_train_scaled)
    y_te_pred = model.predict(X_test_scaled)

    # Metrics
    mse_train = mean_squared_error(y_train, y_tr_pred)
    mse_test  = mean_squared_error(y_test,  y_te_pred)
    r2_train  = r2_score(y_train, y_tr_pred)
    r2_test   = r2_score(y_test,  y_te_pred)
    mae_train = mean_absolute_error(y_train, y_tr_pred)
    mae_test  = mean_absolute_error(y_test,  y_te_pred)

    results.append([
        model_name, mse_train, mse_test, r2_train, r2_test, mae_train, mae_test, mean_cv_mse
    ])

results_df = pd.DataFrame(results, columns=[
    'Model', 'Train MSE', 'Test MSE', 'Train R²', 'Test R²', 'Train MAE', 'Test MAE', 'CV MSE'
])

display(results_df)

Model  Train MSE  Test MSE  Train R²  Test R²  Train MAE  \
0  Linear Regression     0.7590    0.9313    0.7885   0.7922     0.6920   
1         ElasticNet     1.0831    1.5255    0.6982   0.6596     0.8151   
2      Random Forest     0.4894    0.7984    0.8637   0.8218     0.5565   
3           AdaBoost     0.4443    0.8318    0.8762   0.8144     0.5803   
4            XGBoost     0.0129    0.4649    0.9964   0.8962     0.0884   
5   Lasso Regression     1.2086    1.7200    0.6633   0.6162     0.8650   

   Test MAE  CV MSE  
0    0.7743  1.1689  
1    0.9653  1.3881  
2    0.7463  1.2220  
3    0.7475  1.1265  
4    0.5142  1.2216  
5    1.0163  1.4864